## Historical S&P 500 composition

The goal of this script is to create historical S&P500 composition using the table of "Selected changes to the list of S&P 500 components" from wikipedia. Therefore a guarenteed correctness can not be fullfilled. It depends on the correctness of the table

In [1]:
#Import the libraries
import numpy as np
import pandas as pd
import copy
from datetime import datetime, timedelta

In [2]:
#Reading in the 2 tables from wikipedia
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
sp_today = data[0] #table of SP500 today
sp_changes = data[1] #tables of changes since 1997

In [3]:
#Setting the head of the table right
sp_changes.columns = ["Date", "Added_Ticker", "Added_Security", "Removed_Ticker", "Removed_Security", "Reason"]

In [4]:
#Changing the Date columns to a Datetime object
sp_changes["Date"] = pd.to_datetime(sp_changes["Date"])
#fill all the Nan values with 0 for easier handling
sp_changes.fillna("0", inplace=True)

In [5]:
#See the last date of recorded changes in the changes table
list(sp_changes["Date"])[-1]

Timestamp('1999-12-07 00:00:00')

In [6]:
#creating a variable for the last date and today
last_date = list(sp_changes["Date"])[-1].date()
today = datetime.now().date()

In [7]:
#creating a variable with all the tickers in the sp500 today
tickers_today = list(sp_today["Symbol"])

In [8]:
#Create an empty dictionary to fill with dates and tickers
tickers_on_date = {}
today = pd.to_datetime(datetime.today().date())
tickers_today = list(sp_today["Symbol"])
tickers_updated = tickers_today

for ticker in sp_changes.values:
    tickers_today = tickers_updated
    while today != ticker[0]:
        tickers_on_date[str(today)] = tickers_today
        today -= timedelta(1)
    if ticker[0] == last_date:
        break
    elif ticker[1] == "0":
        tickers_updated = copy.deepcopy(tickers_today)
        tickers_updated.append(ticker[3])
    elif ticker[3] == "0":
        tickers_updated = copy.deepcopy(tickers_today)
        tickers_updated.remove(ticker[1])

    try:
        tickers_updated = copy.deepcopy(tickers_today)
        tickers_updated.append(ticker[3])
        tickers_updated.remove(ticker[1])
    except ValueError:
        continue
    tickers_on_date[str(today)] = tickers_updated


C:\Users\Han\AppData\Local\Temp/ipykernel_6328/2339500110.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if ticker[0] == last_date:


In [9]:
#Test out if our dictionary with historical sp500 composition is right
def test_true(ticker_name, timestamp):
    if ticker_name in tickers_on_date[timestamp]:
        print(ticker_name, "is in the S&P500 at the timestamp: ", timestamp)
    else:
        print(ticker_name, "is not in the S&P500 at the timestamp: ", timestamp)

In [10]:
test_true("FB", "2013-12-24 00:00:00")

FB is in the S&P500 at the timestamp:  2013-12-24 00:00:00


In [11]:
#tickers_on_date.keys()